In [1]:
import sys
sys.path.append('../')

from common_imports import *
from common_functions import *

In [2]:
df = pd.read_csv("lookup.csv")

In [3]:
df.columns

Index(['name', 'datetime', 'temp', 'dew', 'precip', 'precipcover', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure',
       'cloudcover', 'solarradiation', 'Lat', 'Long', 'county', 'elevation',
       'Started', 'Duration', 'OptimizedSeverityScore',
       'OptimizedSeverityScore_Log', 'closest_city'],
      dtype='object')

In [4]:
df.head(5)

,name,datetime,temp,dew,precip,precipcover,snow,snowdepth,windgust,windspeed,...,solarradiation,Lat,Long,county,elevation,Started,Duration,OptimizedSeverityScore,OptimizedSeverityScore_Log,closest_city
0,"Susanville, CA",2018-04-01,11.1,-1.1,0.0,0.00,0.0,0.7,43.2,31.7,...,273.9,40.4163,-120.653,Lassen,1280,NaN,NaN,NaN,NaN,NaN
1,"Susanville, CA",2018-04-02,10.9,-4.4,0.0,0.00,0.0,0.3,41.8,27.7,...,280.7,40.4163,-120.653,Lassen,1280,NaN,NaN,NaN,NaN,NaN
2,"Susanville, CA",2018-04-03,7.1,-6.4,0.0,0.00,0.0,0.2,39.6,20.5,...,256.5,40.4163,-120.653,Lassen,1280,NaN,NaN,NaN,NaN,NaN
3,"Susanville, CA",2018-04-04,13.2,-3.2,0.2,8.33,0.0,0.1,31.7,18.4,...,189.3,40.4163,-120.653,Lassen,1280,NaN,NaN,NaN,NaN,NaN
4,"Susanville, CA",2018-04-05,10.9,-0.1,2.1,25.00,0.0,0.0,47.9,22.3,...,151.0,40.4163,-120.653,Lassen,1280,NaN,NaN,NaN,NaN,NaN


In [5]:
df, weather_features = preprocess_df(df)

In [6]:
df['county'].unique()

array(['Modoc', 'Contra Costa', 'Kern', 'San Bernardino', 'Fresno',
       'Inyo', 'San Diego', 'Riverside', 'Del Norte', 'Imperial',
       'Sacramento', 'Humboldt', 'Los Angeles', 'Stanislaus', 'Ventura',
       'San Luis Obispo', 'Shasta', 'Placer', 'Monterey', 'Santa Barbara',
       'Sonoma', 'El Dorado', 'Lassen', 'Mendocino'], dtype=object)

In [9]:
len(df['county'].unique())

24

In [76]:
df.shape

(128743, 17)

In [77]:
count_check(df)

   FireOccurred  Count
0             0  58295
1             1  70448


In [78]:
cities =  pd.read_csv("weather_data/city_location.csv")

In [79]:
print(len(df["name"].unique()))
print(len(cities["name"].unique()))


44
44


In [80]:
train = df[(df['datetime'] < '2018-01-01')]
val = df[(df['datetime'] >= '2018-01-01') & (df['datetime'] < '2019-01-01')]
test = df[(df['datetime'] >= '2019-01-01')]

In [81]:
print(f'num of observations in train: {len(train)}')
print(f'num of observations in val: {len(val)}')
print(f'num of observations in test: {len(test)}')

num of observations in train: 85973
num of observations in val: 32140
num of observations in test: 10630


In [82]:
count_check(train)
count_check(val)
count_check(test)

   FireOccurred  Count
0             0  45003
1             1  40970
   FireOccurred  Count
0             0   3662
1             1  28478
   FireOccurred  Count
0             0   9630
1             1   1000


In [83]:
city_names = df['name'].unique()

for city in city_names:
        print(city)
        city_df = train[train['name'] == city] 
        count_check(city_df)

Alturas, CA
   FireOccurred  Count
0             0    907
1             1   1158
Antioch, CA
   FireOccurred  Count
0             0   1056
1             1    592
Bakersfield, CA
   FireOccurred  Count
0             0    897
1             1   2221
Barstow, CA
   FireOccurred  Count
0             0   1220
Big Creek, CA
   FireOccurred  Count
0             0    657
1             1   2184
Bishop, CA
   FireOccurred  Count
0             0   1108
1             1    114
Chula Vista, CA
   FireOccurred  Count
0             0    983
1             1    903
Coalinga, CA
   FireOccurred  Count
0             0    975
1             1   2462
Concord, CA
   FireOccurred  Count
0             0   1045
1             1    416
Corona, CA
   FireOccurred  Count
0             0    972
1             1    585
Crescent City, CA
   FireOccurred  Count
0             0    816
1             1   1343
Delano, CA
   FireOccurred  Count
0             0    856
1             1   1216
El Centro, CA
   FireOccurred  Count


In [84]:
cities.columns

Index(['name', 'Lat', 'Long', 'county', 'elevation'], dtype='object')

In [85]:
map_center = [37.1661, -119.4494]
map_obj = folium.Map(location=map_center, zoom_start=12)

for _, row in cities.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Long"]],
        radius=5,  # Size of the dot
        color="blue",  # Border color
        fill=True,
        fill_color="blue",  # Fill color
        fill_opacity=0.6
    ).add_to(map_obj)


map_obj

In [86]:
# weather_api_key = 'P6FXD6GPSFYY8WCA67HXNW8JP'
# url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"


In [87]:
# cities = [# "Riverside, CA", "Moreno Valley, CA", "Corona, CA", //
#           # "Los Angeles, CA", "Long Beach, CA", "Glendale, CA", //
#           # "San Bernardino, CA", "Fontana, CA", "Rancho Cucamonga, CA", //
#           # "San Diego, CA", "Chula Vista, CA", "Oceanside, CA" //
#         #   "Bakersfield, CA", "Delano, CA", "Ridgecrest, CA", ///
#         #   "Concord, CA", "Antioch, CA", "Richmond, CA"*, ///
#         #   "Sacramento, CA", "Elk Grove, CA", "Folsom, CA", ///
#         #   "Oxnar, CA", "Thousand Oaks, CA", "Ventura, CA", ///
#         #   "Fresno, CA", "Coalinga, CA", "Big Creek, CA", ///
#         #   "Roseville, CA", "Crescent City, CA", "Redding, CA", ///
#         #   "South Lake Tahoe, CA", "Santa Maria, CA", "Modesto, CA" ///
# ]  # cities of interest

# columns = ["datetime", "latitude", "longitude", 'address', 'tempmax','tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'preciptype',
#             'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure', 'cloudcover', 
#             'visibility', 'solarradiation', 'solarenergy', 'uvindex'] # columns of interest

# output_file = 'ca_county_weather.csv'

In [88]:
# end_date = pd.Timestamp('2014-04-01')
# start_date = pd.Timestamp('2014-11-30')

# dates = pd.date_range(start=start_date, end=end_date, freq='D').strftime('%Y-%m-%d').tolist()

In [89]:
# for city in cities:
#     for date in dates:

#         full_url = f"{url}/{city}/{date}?key={weather_api_key}&include=current"
#         response = requests.get(full_url)

#         data = response.json()

#         weather_data = []

#         for day in data['days']:
#             day_data = {col: day[col] for col in columns if col in day}
#             day_data['location'] = data['address']
#             weather_data.append(day_data)

#         df = pd.DataFrame(weather_data)
#         df.to_csv(output_file, mode='a', header=True, index=True)